In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import *
import numpy as np

In [44]:
def draw_graph(history):

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [45]:
cluster_path = '../csv/preprocessed/clustered_model.csv'
cluster = pd.read_csv(cluster_path)
cluster.head()

,Frequency,Timeinterval-avg,Timeinterval-std,Size-avg,Cluster
0,104,104.177744,599.385970,512.000000,Hot
1,178,61.051303,417.857584,512.000000,Hot
2,128,84.295964,540.087874,512.000000,Hot
3,138,78.143073,413.777300,537.971014,Hot
4,129,83.637812,535.740946,539.782946,Hot


In [46]:
print('Hot : ', len(cluster[cluster['Cluster'] == 'Hot']))
print('Warm : ', len(cluster[cluster['Cluster'] == 'Warm']))
print('Cold : ', len(cluster[cluster['Cluster'] == 'Cold']))

Hot :  29757
Warm :  65928
Cold :  18546


In [47]:
# Encode the labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(cluster['Cluster'])
cluster['Cluster'] = encoded_labels

encoded_dict = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Encoded:", encoded_dict)

# 디코딩 딕셔너리
decoded_dict = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
print("Decoded:", decoded_dict)

# Prepare the data for LSTM
X = cluster.drop(['Cluster'], axis=1).values
y = cluster['Cluster'].values

Encoded: {'Cold': 0, 'Hot': 1, 'Warm': 2}
Decoded: {0: 'Cold', 1: 'Hot', 2: 'Warm'}


In [ ]:
# Reshape X to be 3D [samples, timesteps, cluster] as required for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

reg_l2 = tf.keras.regularizers.l2(0.01)

# Build the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64,
                         input_shape=(X_train.shape[1], X_train.shape[2]),
                         kernel_regularizer=reg_l2,
                         recurrent_regularizer=reg_l2,
                         bias_regularizer=reg_l2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: Hot, Cold, Warm
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping('loss', patience=2, mode='min')


# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])

draw_graph(history)

In [6]:
# f = open("train_log.txt", 'w')
# f.write('loss : ', history.history['loss'])
# f.write('accuracy : ', history.history['accuracy'])
# f.write('val_loss : ', history.history['val_loss'])
# f.write('val_accuracy : ',history.history['val_accuracy'])
# f.close()

fmt = lambda x: '{:.4f}'.format(x)

print('loss : ', list(map(fmt, history.history['loss'])))
print('accuracy : ', list(map(fmt, history.history['accuracy'])))
print('val_loss : ', list(map(fmt, history.history['val_loss'])))
print('val_accuracy : ', list(map(fmt, history.history['val_accuracy'])))

loss :  ['0.6239', '0.5167', '0.5323', '0.5427']
accuracy :  ['0.7417', '0.7448', '0.7500', '0.7372']
val_loss :  ['0.7378', '0.6644', '0.6107', '0.5455']
val_accuracy :  ['0.7747', '0.7128', '0.7625', '0.7793']


In [57]:
def predict_input(input_array):
    input_array = input_array.reshape((input_array.shape[0], 1, input_array.shape[1]))
    probabilities = model.predict(input_array)
    predict_labels = np.argmax(probabilities, axis = 1)
    return decoded_dict[predict_labels[0]]

x = np.array([[1000, 1, 2, 500]])
predict_input(x)

1/1 [==============================] - 0s 16ms/step


'Hot'

In [18]:
# model.save('lstm.h5')